In [21]:
import pandas as pd
from itertools import permutations
from datetime import timedelta


In [7]:
schedule = pd.read_excel("mlb_schedule_2025_for_reddit.xlsx", header=1)
schedule.head()

,Game Date,Day of Week,Local Time,Time - ET,Game,Away Team,Home Team,Location
0,2025-03-18,Tuesday,06:00:00,06:00:00,Dodgers at Cubs,Dodgers,Cubs,Tokyo Dome - Tokyo
1,2025-03-19,Wednesday,06:00:00,06:00:00,Dodgers at Cubs,Dodgers,Cubs,Tokyo Dome - Tokyo
2,2025-03-27,Thursday,15:10:00,16:10:00,Mets at Astros,Mets,Astros,Daikin Park - Houston
3,2025-03-27,Thursday,15:07:00,15:07:00,Orioles at Blue Jays,Orioles,Blue Jays,Rogers Centre - Toronto
4,2025-03-27,Thursday,15:15:00,16:15:00,Twins at Cardinals,Twins,Cardinals,Busch Stadium - St. Louis


In [8]:
def build_games_lookup(schedule):
    rows = []
    for _, row in schedule.iterrows():
        for team, loc in [(row["Home Team"], "Home"), (row["Away Team"], "Away")]:
            rows.append({
                "Team": team,
                "Opponent": row["Away Team"] if team == row["Home Team"] else row["Home Team"],
                "Location": loc,
                "Date": row["Game Date"],
                "Day": row["Day of Week"],
                "Stadium": row["Location"]
            })
    return pd.DataFrame(rows)

games_df = build_games_lookup(schedule)


In [9]:
games_df

,Team,Opponent,Location,Date,Day,Stadium
0,Cubs,Dodgers,Home,2025-03-18,Tuesday,Tokyo Dome - Tokyo
1,Dodgers,Cubs,Away,2025-03-18,Tuesday,Tokyo Dome - Tokyo
2,Cubs,Dodgers,Home,2025-03-19,Wednesday,Tokyo Dome - Tokyo
3,Dodgers,Cubs,Away,2025-03-19,Wednesday,Tokyo Dome - Tokyo
4,Astros,Mets,Home,2025-03-27,Thursday,Daikin Park - Houston
...,...,...,...,...,...,...
4855,Twins,Phillies,Away,2025-09-28,Sunday,Citizens Bank Park - Philadelphia
4856,Red Sox,Tigers,Home,2025-09-28,Sunday,Fenway Park - Boston
4857,Tigers,Red Sox,Away,2025-09-28,Sunday,Fenway Park - Boston
4858,Yankees,Orioles,Home,2025-09-28,Sunday,Yankee Stadium - Bronx


In [ ]:
team_stadium_coords = {
    "Diamondbacks": ["Chase Field - Phoenix", (33.4455, -112.0667)],
    "Braves": ["Truist Park - Atlanta", (33.8908, -84.4678)],
    "Orioles": ["Oriole Park at Camden Yards - Baltimore", (39.2839, -76.6218)],
    "Red Sox": ["Fenway Park - Boston", (42.3467, -71.0972)],
    "White Sox": ["Guaranteed Rate Field - Chicago", (41.8299, -87.6338)],
    "Cubs": ["Wrigley Field - Chicago", (41.9484, -87.6553)],
    "Reds": ["Great American Ball Park - Cincinnati", (39.0979, -84.5073)],
    "Guardians": ["Progressive Field - Cleveland", (41.4962, -81.6852)],
    "Rockies": ["Coors Field - Denver", (39.7559, -104.9942)],
    "Tigers": ["Comerica Park - Detroit", (42.3390, -83.0485)],
    "Astros": ["Minute Maid Park - Houston", (29.7573, -95.3555)],
    "Royals": ["Kauffman Stadium - Kansas City", (39.0516, -94.4803)],
    "Angels": ["Angel Stadium - Anaheim", (33.8003, -117.8827)],
    "Dodgers": ["Dodger Stadium - Los Angeles", (34.0739, -118.2400)],
    "Marlins": ["loanDepot Park - Miami", (25.7780, -80.2197)],
    "Brewers": ["American Family Field - Milwaukee", (43.0280, -87.9712)],
    "Twins": ["Target Field - Minneapolis", (44.9817, -93.2773)],
    "Mets": ["Citi Field - New York", (40.7571, -73.8458)],
    "Yankees": ["Yankee Stadium - New York", (40.8296, -73.9262)],
    "Athletics": ["Oakland Coliseum - Oakland", (37.7516, -122.2005)],
    "Phillies": ["Citizens Bank Park - Philadelphia", (39.9057, -75.1665)],
    "Pirates": ["PNC Park - Pittsburgh", (40.4469, -80.0057)],
    "Padres": ["Petco Park - San Diego", (32.7073, -117.1570)],
    "Giants": ["Oracle Park - San Francisco", (37.7786, -122.3893)],
    "Mariners": ["T-Mobile Park - Seattle", (47.5914, -122.3325)],
    "Cardinals": ["Busch Stadium - St. Louis", (38.6226, -90.1928)],
    "Rays": ["Tropicana Field - St. Petersburg", (27.7683, -82.6534)],
    "Rangers": ["Globe Life Field - Arlington", (32.7513, -97.0820)],
    "Blue Jays": ["Rogers Centre - Toronto", (43.6414, -79.3894)],
    "Nationals": ["Nationals Park - Washington", (38.8728, -77.0075)],
}


In [22]:


def find_team_itineraries(
    games_df,
    team_list,
    day_of_week_list=None,
    total_day_span=None,
    home_teams=None,
    away_teams=None
):
    total_day_span = total_day_span or len(team_list)
    filtered_games = games_df[games_df["Team"].isin(team_list)]

    if day_of_week_list:
        filtered_games = filtered_games[filtered_games["Day"].isin(day_of_week_list)]
    if home_teams:
        filtered_games = filtered_games[~(
            (filtered_games["Team"].isin(home_teams)) & (filtered_games["Location"] != "Home")
        )]
    if away_teams:
        filtered_games = filtered_games[~(
            (filtered_games["Team"].isin(away_teams)) & (filtered_games["Location"] != "Away")
        )]

    filtered_games = filtered_games.sort_values("Date")
    valid_itineraries = []
    unique_dates = filtered_games["Date"].unique()

    for start_date in unique_dates:
        end_date = start_date + timedelta(days=total_day_span - 1)
        window = filtered_games[
            (filtered_games["Date"] >= start_date) &
            (filtered_games["Date"] <= end_date)
        ]

        if window["Date"].nunique() < len(team_list):
            continue

        for date_combo in permutations(window["Date"].unique(), len(team_list)):
            itinerary = []
            success = True
            for team, date in zip(team_list, date_combo):
                match = window[(window["Team"] == team) & (window["Date"] == date)]
                if match.empty:
                    success = False
                    break
                row = match.iloc[0]
                itinerary.append({
                    "Team": team,
                    "Date": row["Date"],
                    "Day": row["Day"],
                    "Opponent": row["Opponent"],
                    "Location": row["Location"],
                    "Stadium": row["Stadium"]
                })
            if success:
                valid_itineraries.append({
                    "Start Date": start_date,
                    "End Date": end_date,
                    "Games": itinerary
                })
                break  # only one result per window

    return valid_itineraries


In [23]:
results = find_team_itineraries(
    games_df,
    team_list=["Guardians", "Pirates", "Tigers"],
    day_of_week_list=["Saturday", "Sunday", "Monday"],
    home_teams=["Guardians", "Tigers", "Pirates"],   # optional
    # away_teams=["Tigers"],
    total_day_span= 3       # optional
)


In [19]:


def itineraries_to_dataframe(itineraries):
    all_rows = []

    for itin in itineraries:
        for game in itin["Games"]:
            all_rows.append({
                "Start Date": itin["Start Date"].date(),
                "End Date": itin["End Date"].date(),
                "Team": game["Team"],
                "Date": game["Date"].date(),
                "Day": game["Day"],
                "Opponent": game["Opponent"],
                "Location": game["Location"],
                "Stadium": game["Stadium"]
            })

    return pd.DataFrame(all_rows)


In [33]:
results = find_team_itineraries(
    games_df,
    team_list=["Guardians", "Pirates", "Tigers", "Phillies"],
    day_of_week_list=["Tuesday", "Saturday", "Sunday", "Monday"],
    home_teams=["Guardians", "Tigers", "Pirates", "Phillies"]
)

df_results = itineraries_to_dataframe(results)
df_results.sort_values(by = 'Date', ascending = True).head(50)


,Start Date,End Date,Team,Date,Day,Opponent,Location,Stadium
1,2025-04-05,2025-04-08,Pirates,2025-04-05,Saturday,Yankees,Home,PNC Park - Pittsburgh
3,2025-04-05,2025-04-08,Phillies,2025-04-06,Sunday,Dodgers,Home,Citizens Bank Park - Philadelphia
2,2025-04-05,2025-04-08,Tigers,2025-04-07,Monday,Yankees,Home,Comerica Park - Detroit
0,2025-04-05,2025-04-08,Guardians,2025-04-08,Tuesday,White Sox,Home,Progressive Field - Cleveland
5,2025-04-19,2025-04-22,Pirates,2025-04-19,Saturday,Guardians,Home,PNC Park - Pittsburgh
7,2025-04-19,2025-04-22,Phillies,2025-04-20,Sunday,Marlins,Home,Citizens Bank Park - Philadelphia
4,2025-04-19,2025-04-22,Guardians,2025-04-21,Monday,Yankees,Home,Progressive Field - Cleveland
6,2025-04-19,2025-04-22,Tigers,2025-04-22,Tuesday,Padres,Home,Comerica Park - Detroit
8,2025-05-10,2025-05-13,Guardians,2025-05-10,Saturday,Phillies,Home,Progressive Field - Cleveland
9,2025-05-10,2025-05-13,Pirates,2025-05-11,Sunday,Braves,Home,PNC Park - Pittsburgh


In [2]:
df_dist = pd.read_excel("mlb_distances.xlsx")

# Build distances dictionary
distances_between_stadiums = {}
for _, row in df_dist.iterrows():
    team1 = row['Team 1']
    team2 = row['Team 2']
    miles = row['Distance (miles)']
    # store both (team1, team2) and (team2, team1)
    distances_between_stadiums[(team1, team2)] = miles
    distances_between_stadiums[(team2, team1)] = miles

print(distances_between_stadiums)

{('Arizona Diamondbacks', 'Atlanta Braves'): 1585.94, ('Atlanta Braves', 'Arizona Diamondbacks'): 1585.94, ('Arizona Diamondbacks', 'Baltimore Orioles'): 2003.87, ('Baltimore Orioles', 'Arizona Diamondbacks'): 2003.87, ('Arizona Diamondbacks', 'Boston Red Sox'): 2297.68, ('Boston Red Sox', 'Arizona Diamondbacks'): 2297.68, ('Arizona Diamondbacks', 'Chicago White Sox'): 1452.86, ('Chicago White Sox', 'Arizona Diamondbacks'): 1452.86, ('Arizona Diamondbacks', 'Chicago Cubs'): 1453.98, ('Chicago Cubs', 'Arizona Diamondbacks'): 1453.98, ('Arizona Diamondbacks', 'Cincinnati Reds'): 1580.61, ('Cincinnati Reds', 'Arizona Diamondbacks'): 1580.61, ('Arizona Diamondbacks', 'Cleveland Guardians'): 1749.22, ('Cleveland Guardians', 'Arizona Diamondbacks'): 1749.22, ('Arizona Diamondbacks', 'Colorado Rockies'): 586.06, ('Colorado Rockies', 'Arizona Diamondbacks'): 586.06, ('Arizona Diamondbacks', 'Detroit Tigers'): 1690.23, ('Detroit Tigers', 'Arizona Diamondbacks'): 1690.23, ('Arizona Diamondbacks'